In [ ]:
# Подготовка
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.graphics.mosaicplot import mosaic

import zipfile


__z = zipfile.ZipFile("dataset.zip")
#df_str = __z.open("csgo_round_snapshots.csv").read().decode("utf-8")
df_str_fs = __z.open("csgo_round_snapshots.csv")

df = pd.read_csv(df_str_fs)

category_columns = ["map", "bomb_planted", "round_winner"]

# Предварительная обработка данных

Данные [отсюда](https://www.kaggle.com/datasets/christianlillelund/csgo-round-winner-classification):

![](./imgs/table.png)

In [ ]:
# Look dataframe
#df.head()
pd.set_option('display.max_columns', 97)
df.tail()

# Убираем дубликаты

In [ ]:
print(f"Number of (rows, columns): {df.shape}")
duplicate_rows_df = df[df.duplicated()]
print(f"Number of duplicate (rows, columns): {duplicate_rows_df.shape}")
df = df.drop_duplicates()
print(f"Number of (rows, columns) after drop dublicates: {df.shape}")

# Смотрим, чтобы не было нерелевантных данных

## Как поступить с такими данными?

![](./imgs/if_missing_data.png)

In [ ]:
pd.set_option('display.max_rows', 97)
df.count() # Кол-во не None значений в каждой колонке

In [ ]:
pd.set_option('display.max_rows', 97)
print(df.isnull().sum()) # Смотрим есть ли null хотя бы в каком-нибудь столбце
# null'ов нет, поэтому "df = df.dropna()" не нужно

# Делаем кодирование строк + float->int

In [ ]:
df_float_cat = df.copy()

label_columns = category_columns
for label_column_i in label_columns:
    df_float_cat[label_column_i] = df_float_cat[label_column_i].astype('category')

pd.set_option('display.max_rows', 97)
df_float_cat.dtypes                            # Смотрим какие вообще есть типы

In [ ]:
df_ints = df.copy()

float64_cols = df_ints.loc[:, df_ints.dtypes == float].columns

df_ints[float64_cols] = df_ints[float64_cols].astype(int)

# Label Encoding
label_columns = category_columns
for label_column_i in label_columns:
    df_ints[label_column_i] = df_ints[label_column_i].astype('category')
    df_ints[label_column_i] = df_ints[label_column_i].cat.codes
    df_ints[label_column_i] = df_ints[label_column_i].astype(int)

pd.set_option('display.max_rows', 97)
df_ints.dtypes

# Графики

![](./imgs/which_visualization.png)

В нашем случае есть смысл только в мозаичные диаграммы и диаграммы размаха

In [ ]:
# Тепловая карта


plt.figure(figsize=(100,50))
c= df_ints.corr()
sns.heatmap(c,cmap="BrBG",annot=True)
c

In [ ]:
sns.boxplot(x=df['time_left'])

In [ ]:
# Диаграммы размаха

float64_cols = df_float_cat.loc[:, df_float_cat.dtypes == "float64"].columns

for float_i in float64_cols:
    df_float_cat.boxplot(by ='round_winner', column =[float_i], grid = True)

In [ ]:
# Мозаичные диаграммы

plt.rcParams["figure.figsize"]=(10, 5)
category_cols = df_float_cat.loc[:, df_float_cat.dtypes == "category"].columns

for cat_i in category_cols:
    if(cat_i != "round_winner"):
        null_out = mosaic(df, [cat_i, 'round_winner'])


In [ ]:
# Гистограммы

plt.figure(figsize=(100,50))

cols = df_ints.columns
for col_i in cols:
    df_ints.hist(column=col_i)
#df_float_cat.hist()